In [1]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd


In [14]:
stations=pd.read_csv('station.csv')

In [2]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')
    
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')

In [3]:
import pyspark.sql as pysql

sqlCtx = pysql.SQLContext(sc)

trip_rdd=sc.textFile('trip.csv')
trips_df =pycsv.csvToDataFrame(sqlCtx,trip_rdd)

In [4]:
#el rdd de los trips
trips_data=trips_df.rdd

In [9]:
circular_trips=trips_data.filter(lambda x: x[4]==x[7])\
                        .map(lambda x:((x[3],x[6]),1))\
                        .reduceByKey(lambda x,y: x+y)\
                        .collect()

In [12]:
circular_trips=sorted(circular_trips,lambda x,y: y[1]-x[1])

In [20]:
lat_lon

[[], []]

In [29]:

def get_coordenates(stations_list):
    """in : a list of ((station_name,station_name),int)
        out: a 2-column matrix with the lat and long of the stations
    """
    lat_lon=[[],[]]
    for trip,frec in circular_trips:
        lat=stations.loc[stations.name==trip[0],'lat']
        lon=stations.loc[stations.name==trip[0],'long']
        lat_lon[0].append(lat)
        lat_lon[1].append(lon)
    return lat_lon

In [81]:
import plotly.plotly as py
from plotly.graph_objs import *
mapbox_access_token = 'pk.eyJ1IjoibGVvY2VudHVyaW9uIiwiYSI6ImNqMGxsZnlyMTAwMDIzMm5vbXNidmtyOHEifQ.YqWBUbyhzglm38sJeRFl7A'

cities_coordenates=[((37.6158637,-122.192001),9,'San Francisco Bay'),((37.788925,-122.403452),12,'San Francisco'),
                    ((37.339301,-121.889937),12,'San Jose') ,((37.487682,-122.223492),12,'Redwood City'),
                    ((37.444521,-122.163093),12,'Palo Alto'),((37.389218,-122.081896),12,'Mountain View')]

lat_long=get_coordenates(circular_trips)
number_of_trips= ['{}'.format(frecuency) for trip,frecuency in circular_trips]
data = Data([
    Scattermapbox(
        lat = lat_long[0],
        lon = lat_long[1],
        text =['{} \n {}'.format(trip[0],frecuency) for trip,frecuency in circular_trips],
        mode='markers',
        marker = dict(
            
            color = number_of_trips,
            colorscale = [[0,"rgb(150,150,255)"],[1,"rgb(0, 0, 255)"]],
            opacity = 1,
            size = 9,
        )
    )
])
cities=[]
for coordenates,zoom,text in cities_coordenates:
    city=dict(
        args=[ { 
            'mapbox.center.lat':coordenates[0],
            'mapbox.center.lon':coordenates[1],
            'mapbox.zoom':zoom,
            'annotations[0].text':text
        } ],
        label=text,
        method='relayout'
    )
    cities.append(city)
updatemenus=list([
    dict(
        buttons = cities,
        x=1,
        y=1
    )
])
layout = Layout(
    autosize=False,
    width=700,
    height=800,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=37.6158637,
            lon=-122.192001

        ),
        pitch=0,
        zoom=9
        ),
    )
layout['updatemenus'] = updatemenus
fig = { 'data':data, 'layout':layout }
py.iplot(fig)

In [76]:
stations[stations.name=="Mountain View City Hall"]

,id,name,lat,long,dock_count,city,installation_date
20,27,Mountain View City Hall,37.389218,-122.081896,15,Mountain View,8/16/2013
